In [ ]:
class Datagram:
    def __init__(self, source, destination):
        self.source = source
        self.destination = destination

In [ ]:
class Element:
    def __init__(self, datagram, next):
        self.datagram = datagram
        self.next = next

In [ ]:
class Queue: #(desnecessaria)
    def __init__(self, size):
        self.size = size
        self.quantity = 0
        self.head = None
        self.tail = None

    def isEmpty(self):
        return self.head == None  

    def enqueue(self, datagram):
        if self.quantity < self.size:
            e = Element(datagram, None)
            if self.isEmpty():
                self.head = e
                self.tail = e
            self.tail.next = e
            self.tail = e
            self.quantity += 1
            return True
        return False

    def dequeue(self):
        if not self.isEmpty():
            datagram = self.head.datagram
            self.head = self.head.next
            self.quantity -= 1
            return datagram
        return None

In [ ]:
class Event:
    def __init__(self, instant, destination, datagram):
        self.instant = instant
        self.destination = destination
        self.datagram = datagram
    

In [ ]:
class Network:
    def __init__(self, size):
        self.size = size
        self.quantity = 0
        self.routers = {}

    def add(self, router):
        if self.quantity < self.size:
            if router.address not in self.routers:
                self.routers[router.address] = router
                self.quantity += 1
                return True
        return False